In [1]:
#Importing necessary labraries
import pandas as pd
import numpy as np

### **Loading The Data set**

In [2]:
from google.colab import files
uploaded =files.upload()

Saving Online retail.xlsx to Online retail.xlsx


In [3]:
df = pd.read_excel("Online retail.xlsx", sheet_name="Sheet1",header=None)

In [4]:
df.columns = ['items'] #Assigning a column name to data set to read all the rows without missing

In [5]:
df.head()

,items
0,"shrimp,almonds,avocado,vegetables mix,green gr..."
1,"burgers,meatballs,eggs"
2,chutney
3,"turkey,avocado"
4,"mineral water,milk,energy bar,whole wheat rice..."


### **Data Preprocessing**

In [6]:
 #Converting the column into a list
transactions = df['items'].str.split(',').tolist()

In [7]:
#Now encoding using transaction encoder
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [8]:
pd.set_option('display.max_columns', None) #Setting to max to review the columns

df_encoded.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,bramble,brownies,bug spray,burger sauce,burgers,butter,cake,candy bars,carrots,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolate bread,chutney,cider,clothes accessories,cookies,cooking oil,corn,cottage cheese,cream,dessert wine,eggplant,eggs,energy bar,energy drink,escalope,extra dark chocolate,flax seed,french fries,french wine,fresh bread,fresh tuna,fromage blanc,frozen smoothie,frozen vegetables,gluten free bar,grated cheese,green beans,green grapes,green tea,ground beef,gums,ham,hand protein bar,herb & pepper,honey,hot dogs,ketchup,light cream,light mayo,low fat yogurt,magazines,mashed potato,mayonnaise,meatballs,melons,milk,mineral water,mint,mint green tea,muffins,mushroom cream sauce,napkins,nonfat milk,oatmeal,oil,olive oil,pancakes,parmesan cheese,pasta,pepper,pet food,pickles,protein bar,red wine,rice,salad,salmon,salt,sandwich,shallot,shampoo,shrimp,soda,soup,spaghetti,sparkling water,spinach,strawberries,strong cheese,tea,tomato juice,tomato sauce,tomatoes,toothpaste,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [9]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Columns: 120 entries,  asparagus to zucchini
dtypes: bool(120)
memory usage: 879.2 KB


### **Association Rule Mining**

In [10]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

#Defining a certain range for supporting values and confidence %
support_values = [0.02,0.03,0.05]  #Since it's a large data set
confidence_values = [0.3, 0.4,0.5]
lift_threshold = 1.6  #Let's Keep this constant initially so we adjust it in future

results = [] #Storing results for assigning

for support in support_values:

    frequent_itemsets = apriori(df_encoded, min_support=support, use_colnames=True) ##To find best minimum support

    for confidence in confidence_values:

        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence) #Generating rules for different confidence %
        strong_rules = rules[rules["lift"] >= lift_threshold]

        #Storing the results as summary to understand better
        results.append({
            "Support": support,
            "Confidence": confidence,
            "Num_Rules": len(strong_rules),
            "Avg_Lift": strong_rules["lift"].mean() if not strong_rules.empty else 0,
            "Avg_Confidence": strong_rules["confidence"].mean() if not strong_rules.empty else 0
        })

#Converting results into a DataFrame for analysis.
results_df = pd.DataFrame(results)

results_df


,Support,Confidence,Num_Rules,Avg_Lift,Avg_Confidence
0,0.02,0.3,8,1.870108,0.383714
1,0.02,0.4,3,1.806793,0.430682
2,0.02,0.5,0,0.000000,0.000000
3,0.03,0.3,2,2.019342,0.407734
4,0.03,0.4,1,1.747522,0.416554
5,0.03,0.5,0,0.000000,0.000000
6,0.05,0.3,0,0.000000,0.000000
7,0.05,0.4,0,0.000000,0.000000
8,0.05,0.5,0,0.000000,0.000000


In [11]:
best_support = 0.02
best_confidence = 0.3
lift_threshold = 1.6  #Keeping constant for better rules

frequent_itemsets = apriori(df_encoded, min_support=best_support, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=best_confidence)
strong_rules = rules[rules["lift"] >= lift_threshold]

strong_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(burgers),(eggs),0.087188,0.179709,0.028796,0.330275,1.837830,1.0,0.013128,1.224818,0.499424,0.120941,0.183552,0.245256
4,(cooking oil),(mineral water),0.051060,0.238368,0.020131,0.394256,1.653978,1.0,0.007960,1.257349,0.416672,0.074752,0.204676,0.239354
7,(ground beef),(mineral water),0.098254,0.238368,0.040928,0.416554,1.747522,1.0,0.017507,1.305401,0.474369,0.138413,0.233952,0.294127
8,(ground beef),(spaghetti),0.098254,0.174110,0.039195,0.398915,2.291162,1.0,0.022088,1.373997,0.624943,0.168096,0.272197,0.312015
11,(olive oil),(mineral water),0.065858,0.238368,0.027596,0.419028,1.757904,1.0,0.011898,1.310962,0.461536,0.099759,0.237201,0.267400
14,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,1.0,0.011020,1.401255,0.503221,0.086760,0.286354,0.276610
18,(olive oil),(spaghetti),0.065858,0.174110,0.022930,0.348178,1.999758,1.0,0.011464,1.267048,0.535186,0.105651,0.210764,0.239939
19,(tomatoes),(spaghetti),0.068391,0.174110,0.020931,0.306043,1.757755,1.0,0.009023,1.190117,0.462740,0.094465,0.159746,0.213129


### **Analysis and Interpretations of the Results**


*  Customers who buys ground beef is more likely to buy spaghetti as the confidence % is 39.89% suggest that customer craving for classic bolognese or spicy beef stir fry spagetti .which is supported by 4% transactions with lift of 2.2 means ground beef purchase is has double chances than spagetti.

*  Sales of soup gives almost half of the chance(with support 2.3& lift: 1.91) of buying mineral water means customer going for healthy meal options.

*  Olive oil and spagetti are brought together with confidence of 34% and lift 2.00 means doubles the chances of buying spagetti probably the customers want to eat italian-style meals.

*  Considering confidence of 41 % most of the customers who broght ground beef is very likely to buy mineral water,with support of 4% and lift of 1.74.suggesting there most likely buying for tenderizing the ground beef.

* Burger and eggs are purchased together with having 33% confidence and almost 3% support.it seems like a breakfast meal with scrambled eggs.


### **In Buisness Aspect**


**=> Optimizing Store layouts**

 - Placing the items one after another which makes user to buy on impulse.
(Suggestion placing consequents before antecedents makes customer choose fast since most doesn't have time takes a step back)

 - For instance place pasta and spagetti section before meat section.or mineral water before soups.

**=> Cross Selling & Bundling**

- Offering a special discount on consequent if the customers buy antecedents

- Bundling together(antecedent&consequent) with special price.

- Promoting a new product along side which is useful to the recipes that customer thinking.for example olive oil used in pasta type recipes and salads

**=> Marketing**

- Special offers for those who brought antecedents recently or next time offers like if they buy them together next time there will a special discount.

- Personalized promotions like offering free sample of new product if the customer buys antecedent&consequent together.

- By giving gift cards,lottery cards which promotes them buying together.

### **Recommendations**

- Placing a new product that goes well with what customers purchasing.for instance cutomer who purchase burger & eggs together most likely will choose fries and mashed potatoes.

- And learning about communinty to increaseing certain products they most likely searching in diferent store
 Example:If community have certain type of background like businessmen or working class they have less time to cook so adding quick meal products increase more sales like wise if community has asian they are very likely to choose spices more .

### **Interview Questions**

**Lift and it's importance in Associations rule**

- A specific measurement of two random products brought together by a customers that helps determine their association with each other.

      Formula: Lift = confidence/support of consequent

- => Importance
          
     * A positive association between products means they are brought together more often than a randon chance.

     * As well as a negative association means there just happened to be brought together which is just a random chance.

     * If there's a negative association between items means they were less likely brought together might be less than randon chances.

     Ex: Between ground beef and spagetti there's 2.29 lift meaning spagetti  has double chances buying together with ground beef.

**Support And Confidence And Their Calculation**
  
  - => Support
     
     * The proportion of transactions of the customers where both products are purchased.

           Formula: Support = transaction contain both items/no.of transactions

      Ex: there are 100 transactions contains both A & B in 5000 total transactions so the support values as per formula is 0.02 means 2%.

  - => Confidence
    
    * A customer brought the antecedent but the probability of him/her buying consequent is called confidence probability.

          Formula:  Confidence = Support of both items/support of antecedent

     Ex:Let's say that support of Rice and water is 0.047 and rice is 0.11
     after calculating it's confidence is 0.427 which means it has 43% probability.

**Limitations And Challenges Of Association Rule Mining**

=>  1. Handling Large Data sets

   - It's hard to implement on exremely large data sets.

   - The no.of possible rules grow exponentially as with increase of products.

=>  2. Setting right thresholds
   
   - Choosing right support&confidence values.

   - Setting them too high there's high chance of losing important rules.as well as setting them too low get us weak rules.

=>  3. Unimportant rules

   - Some rules are obvious(Bread and jam/peanut butter) while some are don't work in business just like a rule we just worked on burer can't be processed so do eggs.

   - Non related products associations between food items and accessories,cleaning, processed vs preprocessed etc .are impossible to work on as they bring problems in organizing the products

=> 4. Lack of Causality

 - Just because two products brought together doesn't mean they are associated.

 Ex: A customer purchased whole wheat rice and energy bar together they are completely not associated with each other.

 - Random timings makes random associations probably it maybe the customer's urgency and happened to buy a random product for later use.

=> 5. Problems with rare items

 - Products with low support might be ignored even though they have high confidence.

 Ex: A luxary item may purchased often but it might not have support cuz of less sales
 - Customers can be picky so any imported goods can have high confidence and support too but it lack of supply.
   
